In [47]:
import xgboost as xgb
import pandas as pd
import json
from pathlib import Path
import plotly.express as px

In [2]:
features_file = Path(r"data/features.txt")
with open(features_file, "r") as f:
    features = json.load(f)

In [3]:
df = pd.DataFrame(features)
df.head(5)

,rider_name,age,pcs_score,pcs_ratio,pcs_team,top_5s_category_0,top_10s_category_0,top_15s_category_0,top_20s_category_0,top_5s_category_1,...,top_20s_category_3,top_5s_category_4,top_10s_category_4,top_15s_category_4,top_20s_category_4,top_5s_category_gc,top_10s_category_gc,top_15s_category_gc,top_20s_category_gc,wielerpoule_score
0,ARMSTRONG Lance,30,656,0.1288,5094,0,0,0,0,0,...,0,2,0,1,0,0,0,0,0,109
1,HERAS Roberto,27,657,0.1290,5094,0,0,0,0,0,...,0,7,1,3,0,2,0,0,0,17
2,EKIMOV Viatcheslav,35,376,0.0738,5094,0,0,0,0,2,...,0,2,0,1,0,2,1,0,0,0
3,HAMILTON Tyler,30,214,0.0420,5094,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0
4,HINCAPIE George,28,863,0.1694,5094,1,0,0,0,1,...,0,3,3,1,0,1,1,0,0,0


In [4]:
mean_age = df["age"].loc[df["age"] > 0].mean()
print(len(df["age"].loc[df["age"] == 0]))
df["age"].loc[df["age"] == 0] = mean_age
print(len(df["age"].loc[df["age"] == 0]))

47
0


/var/folders/yy/2phz7y_13_bfz2wdhzcqqypc0000gn/T/ipykernel_73409/2489717391.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["age"].loc[df["age"] == 0] = mean_age


In [5]:
mean_pcs_score = df["pcs_score"].loc[df["pcs_score"] > 0].mean()
print(len(df["pcs_score"].loc[df["pcs_score"] == 0]))
df["pcs_score"].loc[df["pcs_score"] == 0] = mean_pcs_score
print(len(df["pcs_score"].loc[df["pcs_score"] == 0]))

21
0


/var/folders/yy/2phz7y_13_bfz2wdhzcqqypc0000gn/T/ipykernel_73409/3261654386.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["pcs_score"].loc[df["pcs_score"] == 0] = mean_pcs_score


In [6]:
mean_pcs_ratio = df["pcs_ratio"].loc[df["pcs_ratio"] > 0].mean()
print(len(df["pcs_ratio"].loc[df["pcs_ratio"] == 0]))
df["pcs_ratio"].loc[df["pcs_ratio"] == 0] = mean_pcs_ratio
print(len(df["pcs_ratio"].loc[df["pcs_ratio"] == 0]))

23
0


/var/folders/yy/2phz7y_13_bfz2wdhzcqqypc0000gn/T/ipykernel_73409/2484181771.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["pcs_ratio"].loc[df["pcs_ratio"] == 0] = mean_pcs_ratio


In [7]:
mean_pcs_team = df["pcs_team"].loc[df["pcs_team"] > 0].mean()
print(len(df["pcs_team"].loc[df["pcs_team"] == 0]))
df["pcs_team"].loc[df["pcs_team"] == 0] = mean_pcs_team
print(len(df["pcs_team"].loc[df["pcs_team"] == 0]))

16
0


/var/folders/yy/2phz7y_13_bfz2wdhzcqqypc0000gn/T/ipykernel_73409/2542361263.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["pcs_team"].loc[df["pcs_team"] == 0] = mean_pcs_team


Eerst alle rows waar ergens "None" staat weg gooien
En ook voor waar age, pcs_score, pcs_ratio of pcs_team 0 is de mean invullen.

In [8]:
# Wil de wielerpoulescore niet normalizen
columns_to_normalize = list(df.columns)[1:-1]
columns_to_normalize

['age',
 'pcs_score',
 'pcs_ratio',
 'pcs_team',
 'top_5s_category_0',
 'top_10s_category_0',
 'top_15s_category_0',
 'top_20s_category_0',
 'top_5s_category_1',
 'top_10s_category_1',
 'top_15s_category_1',
 'top_20s_category_1',
 'top_5s_category_2',
 'top_10s_category_2',
 'top_15s_category_2',
 'top_20s_category_2',
 'top_5s_category_3',
 'top_10s_category_3',
 'top_15s_category_3',
 'top_20s_category_3',
 'top_5s_category_4',
 'top_10s_category_4',
 'top_15s_category_4',
 'top_20s_category_4',
 'top_5s_category_gc',
 'top_10s_category_gc',
 'top_15s_category_gc',
 'top_20s_category_gc']

In [9]:
for column in columns_to_normalize:
    df[column] = (df[column]-df[column].min())/(df[column].max()-df[column].min())

In [10]:
df.head(5)

,rider_name,age,pcs_score,pcs_ratio,pcs_team,top_5s_category_0,top_10s_category_0,top_15s_category_0,top_20s_category_0,top_5s_category_1,...,top_20s_category_3,top_5s_category_4,top_10s_category_4,top_15s_category_4,top_20s_category_4,top_5s_category_gc,top_10s_category_gc,top_15s_category_gc,top_20s_category_gc,wielerpoule_score
0,ARMSTRONG Lance,0.434783,0.196919,0.128277,0.341678,0.000,0.0,0.0,NaN,0.00,...,NaN,0.08,0.000000,0.052632,NaN,0.000000,0.0,0.0,NaN,109
1,HERAS Roberto,0.304348,0.197221,0.128477,0.341678,0.000,0.0,0.0,NaN,0.00,...,NaN,0.28,0.066667,0.157895,NaN,0.285714,0.0,0.0,NaN,17
2,EKIMOV Viatcheslav,0.652174,0.112353,0.073244,0.341678,0.000,0.0,0.0,NaN,0.10,...,NaN,0.08,0.000000,0.052632,NaN,0.285714,0.2,0.0,NaN,0
3,HAMILTON Tyler,0.434783,0.063425,0.041425,0.341678,0.000,0.0,0.0,NaN,0.00,...,NaN,0.00,0.066667,0.000000,NaN,0.142857,0.0,0.0,NaN,0
4,HINCAPIE George,0.347826,0.259438,0.168901,0.341678,0.125,0.0,0.0,NaN,0.05,...,NaN,0.12,0.200000,0.052632,NaN,0.142857,0.2,0.0,NaN,0


In [11]:
# Drop NaN columns
# Waarschijnlijk zijn die allemaal 0 en daarom NaN geworden
df = df.dropna(axis=1)
df.head(5)

,rider_name,age,pcs_score,pcs_ratio,pcs_team,top_5s_category_0,top_10s_category_0,top_15s_category_0,top_5s_category_1,top_10s_category_1,...,top_5s_category_3,top_10s_category_3,top_15s_category_3,top_5s_category_4,top_10s_category_4,top_15s_category_4,top_5s_category_gc,top_10s_category_gc,top_15s_category_gc,wielerpoule_score
0,ARMSTRONG Lance,0.434783,0.196919,0.128277,0.341678,0.000,0.0,0.0,0.00,0.0,...,0.095238,0.000000,0.0,0.08,0.000000,0.052632,0.000000,0.0,0.0,109
1,HERAS Roberto,0.304348,0.197221,0.128477,0.341678,0.000,0.0,0.0,0.00,0.0,...,0.047619,0.000000,0.0,0.28,0.066667,0.157895,0.285714,0.0,0.0,17
2,EKIMOV Viatcheslav,0.652174,0.112353,0.073244,0.341678,0.000,0.0,0.0,0.10,0.0,...,0.095238,0.083333,0.1,0.08,0.000000,0.052632,0.285714,0.2,0.0,0
3,HAMILTON Tyler,0.434783,0.063425,0.041425,0.341678,0.000,0.0,0.0,0.00,0.0,...,0.142857,0.083333,0.0,0.00,0.066667,0.000000,0.142857,0.0,0.0,0
4,HINCAPIE George,0.347826,0.259438,0.168901,0.341678,0.125,0.0,0.0,0.05,0.0,...,0.190476,0.333333,0.0,0.12,0.200000,0.052632,0.142857,0.2,0.0,0


In [12]:
column_features = list(df.columns)[1:-1]
column_features

['age',
 'pcs_score',
 'pcs_ratio',
 'pcs_team',
 'top_5s_category_0',
 'top_10s_category_0',
 'top_15s_category_0',
 'top_5s_category_1',
 'top_10s_category_1',
 'top_15s_category_1',
 'top_5s_category_2',
 'top_10s_category_2',
 'top_15s_category_2',
 'top_5s_category_3',
 'top_10s_category_3',
 'top_15s_category_3',
 'top_5s_category_4',
 'top_10s_category_4',
 'top_15s_category_4',
 'top_5s_category_gc',
 'top_10s_category_gc',
 'top_15s_category_gc']

In [13]:
column_label = "wielerpoule_score"

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_df, test_df = train_test_split(df, test_size=0.2)

In [16]:
train_target = train_df["wielerpoule_score"]

In [17]:
train_df = train_df.drop(columns=["rider_name", "wielerpoule_score"], axis=1)
train_df.head(5)

,age,pcs_score,pcs_ratio,pcs_team,top_5s_category_0,top_10s_category_0,top_15s_category_0,top_5s_category_1,top_10s_category_1,top_15s_category_1,...,top_15s_category_2,top_5s_category_3,top_10s_category_3,top_15s_category_3,top_5s_category_4,top_10s_category_4,top_15s_category_4,top_5s_category_gc,top_10s_category_gc,top_15s_category_gc
3919,0.478261,0.234068,0.098059,0.535509,0.000,0.000,0.000,0.00,0.000,0.142857,...,0.0,0.238095,0.500000,0.4,0.04,0.266667,0.263158,0.285714,0.2,0.2
2373,0.260870,0.070371,0.056834,0.274739,0.125,0.000,0.000,0.00,0.125,0.285714,...,0.0,0.047619,0.000000,0.0,0.04,0.000000,0.052632,0.000000,0.2,0.2
1686,0.739130,0.413168,0.254953,0.360971,0.125,0.125,0.000,0.10,0.000,0.142857,...,0.0,0.095238,0.166667,0.0,0.36,0.333333,0.105263,0.285714,0.2,0.2
84,0.391304,0.019330,0.013508,0.322523,0.000,0.000,0.000,0.05,0.125,0.000000,...,0.0,0.047619,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.0
2333,0.043478,0.019027,0.008505,0.500380,0.000,0.125,0.125,0.05,0.000,0.142857,...,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.052632,0.000000,0.0,0.2


In [18]:
test_target = test_df["wielerpoule_score"]

In [19]:
test_df = test_df.drop(columns=["rider_name", "wielerpoule_score"], axis=1)
test_df.head(5)

,age,pcs_score,pcs_ratio,pcs_team,top_5s_category_0,top_10s_category_0,top_15s_category_0,top_5s_category_1,top_10s_category_1,top_15s_category_1,...,top_15s_category_2,top_5s_category_3,top_10s_category_3,top_15s_category_3,top_5s_category_4,top_10s_category_4,top_15s_category_4,top_5s_category_gc,top_10s_category_gc,top_15s_category_gc
612,0.434783,0.107520,0.046028,0.523892,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.047619,0.083333,0.0,0.00,0.000000,0.000000,0.000000,0.0,0.0
2603,0.391304,0.412262,0.131879,0.704308,0.0,0.0,0.0,0.10,0.0,0.0,...,0.0,0.190476,0.083333,0.1,0.36,0.266667,0.105263,0.000000,0.2,0.0
1048,0.478261,0.176986,0.092355,0.428670,0.0,0.0,0.0,0.15,0.0,0.0,...,0.0,0.142857,0.250000,0.0,0.00,0.066667,0.000000,0.142857,0.4,0.2
2429,0.304348,0.407128,0.238043,0.381440,0.0,0.0,0.0,0.05,0.0,0.0,...,0.0,0.000000,0.166667,0.0,0.24,0.400000,0.157895,0.428571,0.2,0.0
3523,0.434783,0.041377,0.016310,0.567181,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.04,0.000000,0.052632,0.000000,0.0,0.0


In [145]:
model = xgb.XGBRegressor()

In [146]:
from sklearn.model_selection import GridSearchCV

In [147]:
parameters = {
    "n_estimators": [500, 1000, 2000, 4000],
    "learning_rate": (0.003, 0.01, 0.03, 0.10, 0.3),
    "max_depth": [1, 2, 3, 4, 5, 6, 8],
    "min_child_weight": [ 1, 3, 5, 7],
    "gamma":[0.00, 0.01, 0.03, 0.1, 0.3],
    "colsample_bytree":[ 0.3, 0.4],
    "subsample": [0.6, 0.7, 0.8]
}

In [148]:
gsc = GridSearchCV(
    estimator=model,
    param_grid=parameters,
    cv=3,
    scoring="neg_mean_squared_error",
    verbose=0,
    n_jobs=-1,
)

In [149]:
grid_result = gsc.fit(train_df, train_target)



/Users/meesmolenaar/EigenProjecten/TourDeFrance/venv/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.



In [150]:
best_params = grid_result.best_params_
best_params

{'colsample_bytree': 0.4,
 'gamma': 0.0,
 'learning_rate': 0.003,
 'max_depth': 5,
 'min_child_weight': 7,
 'n_estimators': 2000,
 'subsample': 0.7}

Nog 1x op complete dataset trainen met beste parameters.

In [151]:
final_model = xgb.XGBRegressor(**best_params)

In [152]:
complete_dataset_features = df.drop(columns=["rider_name", "wielerpoule_score"], axis=1)

In [153]:
complete_dataset_target = df["wielerpoule_score"]

In [154]:
final_model.fit(complete_dataset_features, complete_dataset_target)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.4,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0.0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.003, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=7,
             missing=nan, monotone_constraints='()', n_estimators=2000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

In [174]:
final_model_file = Path(r"models/v1_xgboost.pkl")

In [175]:
final_model.save_model(final_model_file)

En dan nu, het moment waarop we hebben gewacht! Het voorspellen :D

In [155]:
last_tour_features_file = Path(r"data/last_tour_features.txt")

In [159]:
with open(last_tour_features_file, "r") as f:
    tour_features = json.load(f)

In [160]:
tour_df = pd.DataFrame(tour_features)
tour_df.head(5)

,rider_name,age,pcs_score,pcs_ratio,pcs_team,top_5s_category_0,top_10s_category_0,top_15s_category_0,top_20s_category_0,top_5s_category_1,...,top_20s_category_3,top_5s_category_4,top_10s_category_4,top_15s_category_4,top_20s_category_4,top_5s_category_gc,top_10s_category_gc,top_15s_category_gc,top_20s_category_gc,wielerpoule_score
0,POGAČAR Tadej,24,1499,0.4750,3156,0,0,0,0,0,...,0,8,1,1,0,3,0,0,0,0
1,BENNETT George,32,45,0.0143,3156,1,1,1,0,0,...,0,0,0,0,0,1,0,3,0,0
2,BJERG Mikkel,24,31,0.0098,3156,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,HIRSCHI Marc,24,425,0.1347,3156,1,1,0,0,0,...,0,1,5,1,0,2,0,1,0,0
4,LAENGEN Vegard Stake,33,44,0.0139,3156,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Die moeten ook even geprocessed

Eerst alle waardes die missen invullen

In [ ]:
mean_age = tour_df["age"].loc[tour_df["age"] > 0].mean()
print(len(tour_df["age"].loc[tour_df["age"] == 0]))
tour_df["age"].loc[tour_df["age"] == 0] = mean_age
print(len(tour_df["age"].loc[tour_df["age"] == 0]))

mean_pcs_score = tour_df["pcs_score"].loc[tour_df["pcs_score"] > 0].mean()
print(len(tour_df["pcs_score"].loc[tour_df["pcs_score"] == 0]))
tour_df["pcs_score"].loc[tour_df["pcs_score"] == 0] = mean_pcs_score
print(len(tour_df["pcs_score"].loc[tour_df["pcs_score"] == 0]))

mean_pcs_ratio = tour_df["pcs_ratio"].loc[tour_df["pcs_ratio"] > 0].mean()
print(len(tour_df["pcs_ratio"].loc[tour_df["pcs_ratio"] == 0]))
tour_df["pcs_ratio"].loc[tour_df["pcs_ratio"] == 0] = mean_pcs_ratio
print(len(tour_df["pcs_ratio"].loc[tour_df["pcs_ratio"] == 0]))

mean_pcs_team = tour_df["pcs_team"].loc[tour_df["pcs_team"] > 0].mean()
print(len(tour_df["pcs_team"].loc[tour_df["pcs_team"] == 0]))
df["pcs_team"].loc[tour_df["pcs_team"] == 0] = mean_pcs_team
print(len(tour_df["pcs_team"].loc[tour_df["pcs_team"] == 0]))

In [163]:
# Drop NaN columns
# Waarschijnlijk zijn die allemaal 0 en daarom NaN geworden
df = df.dropna(axis=1)
df.head(5)

,rider_name,age,pcs_score,pcs_ratio,pcs_team,top_5s_category_0,top_10s_category_0,top_15s_category_0,top_5s_category_1,top_10s_category_1,...,top_5s_category_3,top_10s_category_3,top_15s_category_3,top_5s_category_4,top_10s_category_4,top_15s_category_4,top_5s_category_gc,top_10s_category_gc,top_15s_category_gc,wielerpoule_score
0,ARMSTRONG Lance,0.434783,0.196919,0.128277,0.341678,0.000,0.0,0.0,0.00,0.0,...,0.095238,0.000000,0.0,0.08,0.000000,0.052632,0.000000,0.0,0.0,109
1,HERAS Roberto,0.304348,0.197221,0.128477,0.341678,0.000,0.0,0.0,0.00,0.0,...,0.047619,0.000000,0.0,0.28,0.066667,0.157895,0.285714,0.0,0.0,17
2,EKIMOV Viatcheslav,0.652174,0.112353,0.073244,0.341678,0.000,0.0,0.0,0.10,0.0,...,0.095238,0.083333,0.1,0.08,0.000000,0.052632,0.285714,0.2,0.0,0
3,HAMILTON Tyler,0.434783,0.063425,0.041425,0.341678,0.000,0.0,0.0,0.00,0.0,...,0.142857,0.083333,0.0,0.00,0.066667,0.000000,0.142857,0.0,0.0,0
4,HINCAPIE George,0.347826,0.259438,0.168901,0.341678,0.125,0.0,0.0,0.05,0.0,...,0.190476,0.333333,0.0,0.12,0.200000,0.052632,0.142857,0.2,0.0,0


In [164]:
# Wil de wielerpoulescore niet normalizen
columns_to_normalize = list(tour_df.columns)[1:-1]
columns_to_normalize

['age',
 'pcs_score',
 'pcs_ratio',
 'pcs_team',
 'top_5s_category_0',
 'top_10s_category_0',
 'top_15s_category_0',
 'top_20s_category_0',
 'top_5s_category_1',
 'top_10s_category_1',
 'top_15s_category_1',
 'top_20s_category_1',
 'top_5s_category_2',
 'top_10s_category_2',
 'top_15s_category_2',
 'top_20s_category_2',
 'top_5s_category_3',
 'top_10s_category_3',
 'top_15s_category_3',
 'top_20s_category_3',
 'top_5s_category_4',
 'top_10s_category_4',
 'top_15s_category_4',
 'top_20s_category_4',
 'top_5s_category_gc',
 'top_10s_category_gc',
 'top_15s_category_gc',
 'top_20s_category_gc']

In [166]:
for column in columns_to_normalize:
    tour_df[column] = (tour_df[column]-tour_df[column].min())/(tour_df[column].max()-tour_df[column].min())

In [167]:
trained_features = ['age',
 'pcs_score',
 'pcs_ratio',
 'pcs_team',
 'top_5s_category_0',
 'top_10s_category_0',
 'top_15s_category_0',
 'top_5s_category_1',
 'top_10s_category_1',
 'top_15s_category_1',
 'top_5s_category_2',
 'top_10s_category_2',
 'top_15s_category_2',
 'top_5s_category_3',
 'top_10s_category_3',
 'top_15s_category_3',
 'top_5s_category_4',
 'top_10s_category_4',
 'top_15s_category_4',
 'top_5s_category_gc',
 'top_10s_category_gc',
 'top_15s_category_gc']

In [171]:
df_for_training = tour_df[trained_features]
df_for_training.head()

,age,pcs_score,pcs_ratio,pcs_team,top_5s_category_0,top_10s_category_0,top_15s_category_0,top_5s_category_1,top_10s_category_1,top_15s_category_1,...,top_15s_category_2,top_5s_category_3,top_10s_category_3,top_15s_category_3,top_5s_category_4,top_10s_category_4,top_15s_category_4,top_5s_category_gc,top_10s_category_gc,top_15s_category_gc
0,0.111111,1.000000,0.766645,0.761216,0.00,0.00,0.000000,0.0,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.142857,0.571429,0.1,0.2,0.75,0.0,0.00
1,0.555556,0.026774,0.018513,0.761216,0.25,0.25,0.166667,0.0,0.000000,0.333333,...,0.428571,0.000000,0.000000,0.142857,0.000000,0.0,0.0,0.25,0.0,0.75
2,0.111111,0.017403,0.011205,0.761216,0.00,0.00,0.000000,0.0,0.000000,0.000000,...,0.000000,0.142857,0.166667,0.000000,0.071429,0.0,0.0,0.00,0.0,0.00
3,0.111111,0.281124,0.214031,0.761216,0.25,0.25,0.000000,0.0,0.000000,0.000000,...,0.142857,0.571429,0.500000,0.000000,0.071429,0.5,0.2,0.50,0.0,0.25
4,0.611111,0.026104,0.017863,0.761216,0.00,0.00,0.000000,0.0,0.166667,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00,0.0,0.00


In [172]:
tour_df["prediction"] = final_model.predict(df_for_training)

In [173]:
tour_df.head(5)

,rider_name,age,pcs_score,pcs_ratio,pcs_team,top_5s_category_0,top_10s_category_0,top_15s_category_0,top_20s_category_0,top_5s_category_1,...,top_5s_category_4,top_10s_category_4,top_15s_category_4,top_20s_category_4,top_5s_category_gc,top_10s_category_gc,top_15s_category_gc,top_20s_category_gc,wielerpoule_score,prediction
0,POGAČAR Tadej,0.111111,1.000000,0.766645,0.761216,0.00,0.00,0.000000,NaN,0.0,...,0.571429,0.1,0.2,NaN,0.75,0.0,0.00,NaN,0,70.501328
1,BENNETT George,0.555556,0.026774,0.018513,0.761216,0.25,0.25,0.166667,NaN,0.0,...,0.000000,0.0,0.0,NaN,0.25,0.0,0.75,NaN,0,0.817355
2,BJERG Mikkel,0.111111,0.017403,0.011205,0.761216,0.00,0.00,0.000000,NaN,0.0,...,0.071429,0.0,0.0,NaN,0.00,0.0,0.00,NaN,0,5.446099
3,HIRSCHI Marc,0.111111,0.281124,0.214031,0.761216,0.25,0.25,0.000000,NaN,0.0,...,0.071429,0.5,0.2,NaN,0.50,0.0,0.25,NaN,0,24.234190
4,LAENGEN Vegard Stake,0.611111,0.026104,0.017863,0.761216,0.00,0.00,0.000000,NaN,0.0,...,0.000000,0.0,0.0,NaN,0.00,0.0,0.00,NaN,0,0.960067


In [182]:
predictions_file = Path(r"data/predictions.txt")

In [183]:
tour_df.to_json(predictions_file)